<a href="https://colab.research.google.com/github/MaxVieiraSantiago/Machine-Learning-Algorithms/blob/master/Linear%20Regression/Linear_Regression_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Regressão Linear**

In [74]:
import pandas as pd

In [75]:
file = 'raw_fighter_details.csv'
df = pd.read_csv( file )

In [76]:
df.columns

Index(['fighter_name', 'Height', 'Weight', 'Reach', 'Stance', 'DOB', 'SLpM',
       'Str_Acc', 'SApM', 'Str_Def', 'TD_Avg', 'TD_Acc', 'TD_Def', 'Sub_Avg'],
      dtype='object')

In [77]:
df_selected = df[ ['Height', 'Weight', 'Reach', 'Stance']]
df_selected = df_selected.dropna()

In [78]:
df_selected

,Height,Weight,Reach,Stance
2,"6' 3""",235 lbs.,"76""",Orthodox
6,"5' 11""",170 lbs.,"71""",Orthodox
8,"6' 0""",205 lbs.,"74""",Orthodox
11,"6' 5""",265 lbs.,"80""",Orthodox
12,"6' 1""",185 lbs.,"76""",Orthodox
...,...,...,...,...
3581,"5' 4""",125 lbs.,"66""",Switch
3582,"6' 1""",155 lbs.,"75""",Orthodox
3586,"5' 6""",145 lbs.,"68""",Southpaw
3591,"5' 8""",155 lbs.,"69""",Orthodox


In [79]:
df_selected['Stance'][2]

'Orthodox'

In [80]:
def convert_height(height):
  feet, inches = height.split("\'")
  inches, _ = inches.split('"')

  return float(feet) * 12 + float(inches)

In [81]:
height_series = df_selected['Height']
weight_series = df_selected['Weight']
reach_series = df_selected['Reach']
stance_series = df_selected['Stance']

height = [convert_height(i) for i in height_series]
# [0] pois retorna uma lista com 2 elementos [0, 1]
weight = [float(w.split(' ')[0]) for w in weight_series]
reach = [float(r.split('"')[0]) for r in reach_series]
stance = [-1 if s == 'Orthodox' else 1 for s in stance_series]

In [82]:
data = {
    'Height' : height,
    'Weight' : weight,
    'Reach' : reach,
    'Stance' : stance
}

df_data = pd.DataFrame( data )
df_data.head()

,Height,Weight,Reach,Stance
0,75.0,235.0,76.0,-1
1,71.0,170.0,71.0,-1
2,72.0,205.0,74.0,-1
3,77.0,265.0,80.0,-1
4,73.0,185.0,76.0,-1
